<a href="https://colab.research.google.com/github/TEAM-N0VA/strat-final/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# =============================================================================
# [1단계] 환경 설정 및 추천 데이터(CSV) 업로드
# =============================================================================
# 라이브러리 설치
!pip install flask pyngrok scikit-learn pandas numpy

import pandas as pd
import numpy as np
import io
from google.colab import files

# 전역 변수 초기화 (2단계 서버에서 사용)
global REC_LIST
REC_LIST = []

print("📂 CSV 파일을 업로드해주세요.")
print("   (취소시 추천 기능은 비활성화되고 분석만 수행됩니다)")

try:
    uploaded = files.upload()

    if uploaded:
        filename = next(iter(uploaded))
        print(f"📄 파일명: {filename} 읽는 중...")

        try:
            # 인코딩 호환성 체크 (utf-8 or cp949)
            try:
                sub_db = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='utf-8-sig')
            except:
                sub_db = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='cp949')

            # 비용 데이터 전처리 (콤마/문자 제거 후 숫자로)
            if 'cost' in sub_db.columns:
                sub_db['cost'] = sub_db['cost'].astype(str).str.replace(r'[^0-9]', '', regex=True)
                sub_db['cost'] = pd.to_numeric(sub_db['cost'], errors='coerce').fillna(0)

            # 딕셔너리 형태로 메모리에 로드
            REC_LIST = sub_db.to_dict('records')
            print(f"✅ CSV 로드 성공! {len(REC_LIST)}개의 추천 데이터가 준비되었습니다.")

        except Exception as e:
            print(f"❌ CSV 파일 읽기 실패: {e}")
            REC_LIST = []
    else:
        print("⚠️ 파일 업로드가 취소되었습니다.")

except Exception as e:
    print(f"❌ 업로드 중 에러 발생: {e}")
    REC_LIST = []

print("\n🔽 [완료] 이제 아래의 '2단계: AI 서버 실행' 코드를 실행하세요!")

📂 [1/2] 구독 추천용 CSV 파일을 업로드해주세요.
   (취소시 추천 기능은 비활성화되고 분석만 수행됩니다)


Saving DB - subscription_db.csv to DB - subscription_db (6).csv
📄 파일명: DB - subscription_db (6).csv 읽는 중...
✅ CSV 로드 성공! 88개의 추천 데이터가 준비되었습니다.

🔽 [완료] 이제 아래의 '2단계: AI 서버 실행' 코드를 실행하세요!


In [21]:
# =========================================================
# [1] 필수 라이브러리 설치
# =========================================================
!pip install flask-ngrok flask pyngrok pandas scikit-learn

import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

# =========================================================
# [2] 헬퍼 함수
# =========================================================
def format_usage(value, category):
    """카테고리에 따라 시간(분) 또는 횟수(회)로 포맷팅"""
    if category in ['SHOPPING', 'FOOD']:
        return f"{int(value)}회"
    else:
        minutes = int(value)
        if minutes < 60: return f"{minutes}분"
        hours = minutes // 60
        mins = minutes % 60
        if mins == 0: return f"{hours}시간"
        return f"{hours}시간 {mins}분"

def generate_population_data(n=1000):
    """
    AI 학습용 가상 데이터 (요청 반영 수정)
    - 쇼핑/배달(횟수 기반) 데이터 대폭 하향 조정
    - Normal(평균) 구간을 사용자가 요청한 '6~9회' 수준으로 맞춤
    """
    np.random.seed(42)

    n_heavy = int(n * 0.15)   # 상위 15%
    n_normal = int(n * 0.55)  # 중위 55%
    n_waste = int(n * 0.30)   # 하위 30%

    # 1. 시간 기반 (OTT, Music)
    pop_time_heavy = pd.DataFrame({'usage_amount': np.random.normal(3000, 600, n_heavy), 'category': 'OTT'})
    pop_time_normal = pd.DataFrame({'usage_amount': np.random.normal(1200, 300, n_normal), 'category': 'OTT'})
    pop_time_waste = pd.DataFrame({'usage_amount': np.random.randint(0, 300, n_waste), 'category': 'OTT'})

    # 2. 횟수 기반 (Shopping, Food)
    # Heavy: 10~15회
    pop_count_heavy = pd.DataFrame({'usage_amount': np.random.randint(10, 16, n_heavy), 'category': 'SHOPPING'})

    # Normal: 6~9회
    pop_count_normal = pd.DataFrame({'usage_amount': np.random.randint(6, 10, n_normal), 'category': 'SHOPPING'})

    # Waste: 0~3회
    pop_count_waste = pd.DataFrame({'usage_amount': np.random.randint(0, 4, n_waste), 'category': 'SHOPPING'})

    pop = pd.concat([
        pop_time_heavy, pop_time_normal, pop_time_waste,
        pop_count_heavy, pop_count_normal, pop_count_waste
    ], ignore_index=True)

    pop['usage_amount'] = pop['usage_amount'].apply(lambda x: max(0, x))
    pop['cost'] = 10000

    return pop

# =========================================================
# [3] 핵심 AI 분석 로직
# =========================================================
def analyze_advanced_ai(user_logs_df):
    if user_logs_df.empty: return create_empty_response()

    cols = ['cost', 'timeMinutes', 'payment_count']
    for c in cols:
        user_logs_df[c] = pd.to_numeric(user_logs_df[c], errors='coerce').fillna(0)

    # 1. 파이차트용
    pie_df = user_logs_df.copy()

    # 2. 효율 분석용
    target_subscriptions = ["넷플릭스", "유튜브 프리미엄", "멜론", "배민클럽", "쿠팡와우", "티빙", "디즈니플러스", "요기요"]
    mask = user_logs_df['serviceName'].isin(target_subscriptions)
    sub_raw_df = user_logs_df[mask].copy()

    if sub_raw_df.empty:
        return create_response(pie_df, [], None, {}, "분석할 구독 서비스가 없습니다.")

    # 3. 데이터 집계
    my_agg_df = sub_raw_df.groupby(['serviceName', 'category']).agg({
        'cost': 'sum', 'timeMinutes': 'sum', 'payment_count': 'sum'
    }).reset_index()

    my_agg_df['usage_amount'] = my_agg_df.apply(
        lambda x: x['payment_count'] if x['category'] in ['SHOPPING', 'FOOD'] else x['timeMinutes'], axis=1
    )

    # --- AI 모델링 ---
    pop_df = generate_population_data()
    features = ['usage_amount']

    pop_df['is_user'] = False
    my_agg_df['is_user'] = True
    combined = pd.concat([pop_df[features + ['is_user']], my_agg_df[features + ['is_user']]], ignore_index=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(combined[features])

    km = KMeans(n_clusters=3, random_state=42, n_init=10)
    combined['cluster'] = km.fit_predict(X_scaled)

    c_means = combined.groupby('cluster')['usage_amount'].mean().sort_values()
    sorted_clusters = c_means.index.tolist()

    iso = IsolationForest(contamination=0.05, random_state=42)
    combined['anomaly'] = iso.fit_predict(X_scaled)

    my_results = combined[combined['is_user'] == True].copy()
    my_results.reset_index(drop=True, inplace=True)
    my_agg_df.reset_index(drop=True, inplace=True)
    final_df = pd.concat([my_agg_df, my_results[['cluster', 'anomaly']]], axis=1)

    # 평균 계산
    pop_only = combined[combined['is_user'] == False]
    avg_time_based = pop_only[pop_only['usage_amount'] > 100]['usage_amount'].mean()
    avg_count_based = pop_only[pop_only['usage_amount'] <= 100]['usage_amount'].mean()

    monthly_report = []
    realtime_status = {}

    for i, row in final_df.iterrows():
        usage = row['usage_amount']
        cat = row['category']

        # 1. 평균 비교
        safe_avg = avg_count_based if cat in ['SHOPPING', 'FOOD'] else avg_time_based
        if safe_avg == 0: safe_avg = 1
        pct_diff = int(((usage - safe_avg) / safe_avg) * 100)

        comp_text = f"평균 대비 {abs(pct_diff)}% " + ("상회" if pct_diff >= 0 else "하회")

        # 2. 클러스터 랭크 (0:하위, 1:중위, 2:상위)
        rank = sorted_clusters.index(row['cluster'])

        # 보정 로직
        if pct_diff > 100: rank = 2
        elif pct_diff >= 0 and rank == 0: rank = 1

        # 3. 4단계 등급 결정
        status = "합리적 소비"
        alert = "GOOD"

        limit_red_time = 60; limit_yellow_time = 300
        limit_red_count = 2; limit_yellow_count = 5

        is_red = False; is_yellow = False

        if cat in ['SHOPPING', 'FOOD']:
            if usage < limit_red_count: is_red = True
            elif usage < limit_yellow_count: is_yellow = True
        else:
            if usage < limit_red_time: is_red = True
            elif usage < limit_yellow_time: is_yellow = True

        if is_red:
            status = "🔴 매우 비효율"; alert = "WARNING"; comp_text = "사용 이력 거의 없음"
        elif is_yellow:
            status = "🟡 관심 필요"; alert = "CAUTION"; comp_text = "활용도 다소 낮음"
        elif rank == 1 or (not is_red and not is_yellow and rank == 0):
            status = "🟢 합리적 소비"; alert = "GOOD"
        elif rank == 2:
            status = "🔵 최상의 효율"; alert = "BEST"
            if row['anomaly'] == -1: status = "💎 상위 1% 효율"

        monthly_report.append({
            "service": row['serviceName'],
            "status": status,
            "message": comp_text,
            "detail": f"월 {format_usage(usage, cat)} 사용",
            "alert_level": alert,
            "usage_amount": usage # 정렬을 위해 추가 (JSON 출력엔 제외 가능)
        })

        # 게이지
        target = 1000 if cat not in ['SHOPPING', 'FOOD'] else 10
        realtime_status[row['serviceName']] = {
            "remaining": int(max(target - usage, 0)),
            "unit": "회" if cat in ['SHOPPING', 'FOOD'] else "분",
            "percent": int(min((usage / target) * 100, 100))
        }

    return create_response(pie_df, monthly_report, my_agg_df, realtime_status)

# =========================================================
# [4] 응답 생성 및 추천
# =========================================================
def create_response(raw_df, monthly_report, agg_df, realtime_status, rec_msg=None):
    # 1. 파이차트
    pie_chart = []
    top_category = "NONE"

    if not raw_df.empty:
        grp = raw_df.groupby('category')['cost'].sum().reset_index()
        total = grp['cost'].sum()
        if total > 0:
            grp_sorted = grp.sort_values(by='cost', ascending=False)
            top_category = grp_sorted.iloc[0]['category']
            for i, r in grp_sorted.iterrows():
                pie_chart.append({
                    "category": r['category'],
                    "amount": int(r['cost']),
                    "percent": round(r['cost']/total*100, 1)
                })

    # 2. 페르소나
    persona = "⚖️ 밸런스형 스마트 컨슈머"

    if not raw_df.empty:
        max_row = raw_df.groupby('category')['cost'].sum().sort_values(ascending=False).iloc[0]
        total_cost = raw_df['cost'].sum()
        if (max_row / total_cost) > 0.5:
            cat = raw_df.groupby('category')['cost'].sum().sort_values(ascending=False).index[0]
            if cat == 'OTT': persona = "🍿 콘텐츠 탐험가"
            elif cat == 'FOOD': persona = "🍽️ 맛잘알 미식가"
            elif cat == 'SHOPPING': persona = "🛍️ 쇼핑 마스터"
            elif cat == 'MUSIC': persona = "🎧 감성 리스너"

    # 3. 추천 문구 (다중 비효율 반영 로직)
    if rec_msg is None:
        warning_subs = [x for x in monthly_report if x['alert_level'] == 'WARNING']
        caution_subs = [x for x in monthly_report if x['alert_level'] == 'CAUTION']

        recommendation = ""

        if warning_subs:
            # WARNING이 2개 이상이면 묶어서 보여줌
            if len(warning_subs) >= 2:
                # 서비스명 추출 (최대 2개까지 나열해서 문장 깔끔하게)
                names = ", ".join([w['service'] for w in warning_subs[:2]])
                recommendation = f"현재 '{names}' 모두 사용 효율이 매우 낮습니다. 해지를 고려해보세요."
            else:
                recommendation = f"현재 '{warning_subs[0]['service']}'의 사용 효율이 매우 낮습니다. 구독 유지를 재고해보세요."

        elif caution_subs:
            recommendation = f"'{caution_subs[0]['service']}'의 활용도가 조금 아쉽네요. 혜택을 더 자주 누려보세요!"
        else:
            recommendation = f"{persona}님, 빈틈없는 소비 관리 능력이 돋보입니다! 아주 훌륭해요. 👏"

        rec_msg = recommendation

    # 4. 주간 리포트
    weekly_report = []
    if agg_df is not None and not agg_df.empty:
        for i, row in agg_df.iterrows():
            weekly_avg = int(row['usage_amount'] / 4)
            status_msg = "합리적 사용"; alert = "GOOD"
            for m in monthly_report:
                if m['service'] == row['serviceName']:
                    alert = m['alert_level']
                    if alert == 'WARNING': status_msg = "활용 저조"
                    elif alert == 'CAUTION': status_msg = "관심 필요"
                    elif alert == 'GOOD': status_msg = "합리적 사용"
                    elif alert == 'BEST': status_msg = "최상의 효율"
                    break

            weekly_report.append({
                "service": row['serviceName'],
                "usage_this_week": weekly_avg,
                "message": f"{status_msg} (주 약 {format_usage(weekly_avg, row['category'])})",
                "alert_level": alert
            })

    # 불필요한 키 제거 (JSON 전송용)
    for m in monthly_report:
        if 'usage_amount' in m: del m['usage_amount']

    return {
        "persona": persona,
        "monthly_report": monthly_report,
        "weekly_report": weekly_report,
        "realtime_status": realtime_status,
        "pie_chart": pie_chart,
        "recommendation": rec_msg
    }

def create_empty_response():
    return {
        "persona": "데이터 없음", "monthly_report": [], "weekly_report": [], "realtime_status": {}, "pie_chart": [], "recommendation": "데이터가 없습니다."
    }

# =========================================================
# [5] 서버 실행
# =========================================================
app = Flask(__name__)
# ngrok.set_auth_token("YOUR_TOKEN")

port = 5000
public_url = ngrok.connect(port).public_url
print(f"\n🚀 [AI Server Running] 👉 {public_url}/ \n")

@app.route('/analyze', methods=['POST'])
def analyze():
    try:
        data = request.json
        logs = data.get('logs', [])
        df = pd.DataFrame(logs)
        result = analyze_advanced_ai(df)
        return jsonify(result)
    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(port=port)


🚀 [AI Server Running] 👉 https://hymnodical-superadequately-esmeralda.ngrok-free.dev/ 

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Dec/2025 17:58:03] "POST /analyze HTTP/1.1" 200 -
